In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import zipfile
from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer
from sklearn.model_selection import train_test_split

import sklearn
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Input, GlobalMaxPooling1D, Lambda, Concatenate, Activation, Dropout

import keras.backend as K

In [ ]:
data_normal = pd.read_csv('/content/final_train_data.csv')  # Считывание данных

In [ ]:
!nvidia-smi

Fri May 28 06:08:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
labels = data_normal[['label', 'num_label']]  # Вывод лейблов
labels

,label,num_label
0,normal,2
1,normal,2
2,normal,2
3,normal,2
4,normal,2
...,...,...
5179,guess_passwd,0
5180,guess_passwd,0
5181,guess_passwd,0
5182,guess_passwd,0


In [ ]:
data_normal = data_normal.drop(['label'], axis=1) # Удаление лейблов из обучающего набора

In [ ]:
Y = data_normal['num_label']  # Разделение для обучения
X = data_normal.drop(['num_label'], axis=1)

In [ ]:
scaler = Normalizer().fit(X)  # Нормализация данных
trainX = scaler.transform(X)

In [ ]:
# Разделение на тестовый и обучающие наборы
X_train, X_test, Y_train, Y_test = train_test_split(trainX, Y, test_size = 0.2, shuffle = True)

In [ ]:
# Привидение типов
X_train = np.asarray(X_train).astype('float64')
X_test = np.asarray(X_test).astype('float64')
Y_train = np.asarray(Y_train).astype('float64')
Y_test = np.asarray(Y_test).astype('float64')

In [ ]:
# Привидение лейблов для классификации
Y_train = tf.keras.utils.to_categorical(Y_train, 4)
Y_test = tf.keras.utils.to_categorical(Y_test, 4)

In [ ]:
# Изменение формы данных для подачи на LSTM слой
x_train = X_train.reshape(4147,1,39)
x_test = X_test.reshape(1037, 1, 39)

In [ ]:
model = Sequential()
model.add(LSTM(256,input_dim=39))
model.add(Dropout(0.2))
model.add(Dense(4))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, Y_train, batch_size=32, epochs = 70, validation_split=0.1, verbose = 1)

Epoch 1/70
117/117 [==============================] - 2s 7ms/step - loss: 1.2703 - accuracy: 0.5931 - val_loss: 0.7902 - val_accuracy: 0.7759
Epoch 2/70
117/117 [==============================] - 0s 4ms/step - loss: 0.7143 - accuracy: 0.7006 - val_loss: 0.5962 - val_accuracy: 0.7373
Epoch 3/70
117/117 [==============================] - 0s 4ms/step - loss: 0.5604 - accuracy: 0.7377 - val_loss: 0.5344 - val_accuracy: 0.8096
Epoch 4/70
117/117 [==============================] - 0s 4ms/step - loss: 0.5212 - accuracy: 0.7755 - val_loss: 0.4835 - val_accuracy: 0.8145
Epoch 5/70
117/117 [==============================] - 0s 4ms/step - loss: 0.4794 - accuracy: 0.8368 - val_loss: 0.4359 - val_accuracy: 0.8940
Epoch 6/70
117/117 [==============================] - 0s 4ms/step - loss: 0.4219 - accuracy: 0.8690 - val_loss: 0.3903 - val_accuracy: 0.8867
Epoch 7/70
117/117 [==============================] - 0s 3ms/step - loss: 0.3761 - accuracy: 0.8866 - val_loss: 0.3438 - val_accuracy: 0.9277
Epoch 

In [ ]:
predict = model.predict(x_test)

In [ ]:
loss, accuracy = model.evaluate(x_test, Y_test) # Оценка модели
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

33/33 [==============================] - 0s 2ms/step - loss: 0.0420 - accuracy: 0.9932

Loss: 0.04, Accuracy: 99.32%


In [ ]:
model.save('/content/model')  # Сохранение модели

INFO:tensorflow:Assets written to: /content/model/assets


INFO:tensorflow:Assets written to: /content/model/assets
